# problema caxeiro viajante aluguel de carros 



### Descrição do problema
 Problema do Caixeiro Viajante (TSP) com aluguel de carros é uma extensão do clássico Problema do Caixeiro Viajante, que envolve encontrar a rota mais curta que visita um conjunto de cidades exatamente uma vez e retorna à cidade de origem. No TSP com aluguel de carros, além de visitar as cidades, o caixeiro também pode alugar carros nas cidades para viajar entre elas. O objetivo é minimizar a distância total percorrida, incluindo o aluguel de carros.

Análise de Complexidade:
O TSP com aluguel de carros é um problema NP-difícil. A complexidade computacional do TSP comum já é alta, sendo NP-completo, e a adição do aluguel de carros aumenta a complexidade do problema. A complexidade exata do TSP com aluguel de carros depende de vários fatores, incluindo o número de cidades, o número de carros disponíveis e as restrições de aluguel.


Exact algorithms can find the optimal solution but are only feasible for small problem instances. Examples of exact algorithms include:

- Brute-force search: This involves trying all possible permutations of paths and selecting the shortest one. However, this method quickly becomes impractical even for small instances of the problem due to its factorial time complexity.
- Dynamic programming: The Held-Karp algorithm is an example of a dynamic programming approach to solve the TSP. It has a running time of O(n^2 * 2^n).
- Quantum exact algorithm: More recently, Ambainis et al. proposed a quantum exact algorithm for the TSP that has a running time of O(1.728^n). This algorithm is currently the best-known algorithm for solving the TSP exactly, but it is not yet practical for solving large instances of the problem.
- Branch-and-bound and branch-and-cut algorithms: These algorithms can handle TSPs with 40-60 cities and larger instances respectively. The Concorde TSP Solver, a branch-and-cut algorithm, can solve a TSP with almost 86,000 cities.

https://www.baeldung.com/cs/tsp-exact-solutions-vs-heuristic-vs-approximation-algorithms

# Instance creator

In [1]:
import random 

def generate_symmetric_matrix(size):
    matrix = [[0 for _ in range(size)] for _ in range(size)]
    for i in range(size):
        for j in range(i, size):
            if(i!=j):
                value = random.randint(1, 100)  # Random integer between 1 and 100
                matrix[i][j] = value
                matrix[j][i] = value  # Make it symmetric
    return matrix

def generate_cars_instance_file(citys, cars, file_name):
    with open(file_name, 'w') as file:
        file.write(f"NAME : Test{citys}n\n")
        file.write("TYPE : CaRS\n")
        file.write("COMMENT : Instances for the CaRS Problem (Asconavieta & Goldbarg)\n")
        file.write(f"DIMENSION : {citys}\n")
        file.write(f"CARS_NUMBER : {cars}\n")
        file.write("EDGE_WEIGHT_TYPE : EXPLICIT\n")
        file.write("EDGE_WEIGHT_FORMAT : FULL_MATRIX\n")

        file.write("EDGE_WEIGHT_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")


        file.write("RETURN_RATE_SECTION\n")
        for i in range(cars):
            file.write(f"{i}\n")
            matrix = generate_symmetric_matrix(citys)
            for row in matrix:
                for n in row:
                    file.write(f"{n} ")
                file.write("\n")

        file.write("EOF\n")

# Example usage:
generate_cars_instance_file(6,2, "teste.car")


# Instance reader

In [2]:
class CaRSInstance:
    def __init__(self):
        self.name = ""
        self.type = ""
        self.comment = ""
        self.dimension = 0
        self.cars_number = 0
        self.edge_weight_type = ""
        self.edge_weight_format = ""

class CaRSCar:
    def __init__(self, car_id):
        self.car_id = car_id
        self.distance_matrix = []
        self.return_rate_matrix = []

def read_cars_instance(file_path):
    cars_instance = CaRSInstance()
    cars = []  # List to store Car objects

    with open(file_path, 'r') as file:
        lines = file.readlines()

    section = None
    car_id = None  # To keep track of the current car
    edge_weight_section = False
    return_rate_section = False

    for line in lines:
        line = line.strip()
        if not line:
            continue

        if line.startswith("EDGE_WEIGHT_SECTION"):
            edge_weight_section = True
            section = "EDGE_WEIGHT"
            continue
        elif line.startswith("RETURN_RATE_SECTION"):
            return_rate_section = True
            edge_weight_section = False
            section = "RETURN_RATE"
            continue
        elif line.startswith("EOF"):
            section = None  # Mark the end of the section
            continue

        if edge_weight_section:
            if line.isdigit():
                car_id = int(line)
                cars.append(CaRSCar(car_id))
            else:
                values = line.split()
                cars[-1].distance_matrix.append([int(value) for value in values])
        elif return_rate_section:
            if line.isdigit():
                car_id = int(line)
            else:
                values = line.split()
                cars[car_id].return_rate_matrix.append([int(value) for value in values])

    for line in lines:
        key_value = line.split(":")
        if len(key_value) == 2:
            key, value = key_value
            key = key.strip()
            value = value.strip()

            if key == "NAME":
                cars_instance.name = value
            elif key == "TYPE":
                cars_instance.type = value
            elif key == "COMMENT":
                cars_instance.comment = value
            elif key == "DIMENSION":
                cars_instance.dimension = int(value)
            elif key == "CARS_NUMBER":
                cars_instance.cars_number = int(value)
            elif key == "EDGE_WEIGHT_TYPE":
                cars_instance.edge_weight_type = value
            elif key == "EDGE_WEIGHT_FORMAT":
                cars_instance.edge_weight_format = value

    return cars_instance, cars

# Usage example
file_path = "teste.car"  # Replace with the path to your input file
# file_path = "/Users/tarsila/Documents/paa/PCA-main 2/CaRS_NaoEuclidianas/simple.car"  # Replace with the path to your input file
# file_path = "/Users/tarsila/Documents/paa/PCA-main 2/CaRS_NaoEuclidianas/Bolivia10n.car"  # Replace with the path to your input file
cars_instance, cars = read_cars_instance(file_path)

# Access the parsed data
print("Name:", cars_instance.name)
print("Type:", cars_instance.type)
print("Comment:", cars_instance.comment)
print("Dimension:", cars_instance.dimension)
print("Cars Number:", cars_instance.cars_number)
print("Edge Weight Type:", cars_instance.edge_weight_type)
print("Edge Weight Format:", cars_instance.edge_weight_format)


Name: Test6n
Type: CaRS
Comment: Instances for the CaRS Problem (Asconavieta & Goldbarg)
Dimension: 6
Cars Number: 2
Edge Weight Type: EXPLICIT
Edge Weight Format: FULL_MATRIX


In [3]:

# Access the distance matrices for each car
for car in cars:
    print(f"Car {car.car_id} Distance Matrix:")
    for row in car.distance_matrix:
        print(row)

# Access the return rate matrices for each car
for car in cars:
    print(f"Car {car.car_id} Return Rate Matrix:")
    for row in car.return_rate_matrix:
        print(row)

Car 0 Distance Matrix:
[0, 91, 96, 95, 16, 26]
[91, 0, 53, 94, 31, 91]
[96, 53, 0, 76, 78, 28]
[95, 94, 76, 0, 82, 11]
[16, 31, 78, 82, 0, 29]
[26, 91, 28, 11, 29, 0]
Car 1 Distance Matrix:
[0, 69, 20, 54, 79, 83]
[69, 0, 14, 52, 96, 49]
[20, 14, 0, 10, 24, 2]
[54, 52, 10, 0, 75, 60]
[79, 96, 24, 75, 0, 31]
[83, 49, 2, 60, 31, 0]
Car 0 Return Rate Matrix:
[0, 83, 88, 21, 50, 71]
[83, 0, 35, 39, 25, 65]
[88, 35, 0, 33, 69, 30]
[21, 39, 33, 0, 5, 32]
[50, 25, 69, 5, 0, 66]
[71, 65, 30, 32, 66, 0]
Car 1 Return Rate Matrix:
[0, 8, 19, 38, 72, 39]
[8, 0, 35, 93, 3, 30]
[19, 35, 0, 51, 11, 64]
[38, 93, 51, 0, 8, 16]
[72, 3, 11, 8, 0, 39]
[39, 30, 64, 16, 39, 0]


## generate_random_neighbor

This code defines a function called `generate_random_neighbor` that takes in two parameters: `route` and `car_assignments`. 

The purpose of this function is to generate a random neighbor for a given route and car assignments. 

First, the function creates a copy of the `route` and `car_assignments` lists using the `copy()` and `[:]` methods, respectively. This is done to avoid modifying the original lists.

Next, the function determines the number of cities in the route using the `len()` function and assigns it to the variable `num_cities`.

Then, the function generates a random number of swaps to be performed on the route. The number of swaps is determined by taking the minimum of a random integer between 1 and half of the number of cities, and the difference between the number of cities and 2. This ensures that the number of swaps is within a valid range.

The function then enters a loop that iterates `num_swaps` times. In each iteration, it generates a range of indices from 1 to `num_cities - 1` and assigns it to the variable `r`. 

Next, the function checks if the index 2 is within the range `r`. If it is, it selects two random cities from the range using the `random.sample()` function and assigns them to the variables `city1` and `city2`.

The function then swaps the positions of `city1` and `city2` in the `new_route` list using tuple assignment.

Similarly, it swaps the car assignments for `city1` and `city2` in the `new_car_assignments` list.

After the loop completes, the function returns the modified `new_route` and `new_car_assignments` lists as a tuple.

In [4]:
# Função para gerar um vizinho aleatório
def generate_random_neighbor(route, car_assignments):
    new_route = route.copy()
    new_car_assignments = car_assignments[:]  

    num_cities = len(route)
    num_swaps = min(random.randint(1, num_cities // 2), num_cities - 2)

    for _ in range(num_swaps):
        r = range(1, num_cities - 1)
        if 0 <= 2 <= len(r):
            city1, city2 = random.sample(r, 2)
            new_route[city1], new_route[city2] = new_route[city2], new_route[city1]

            car1, car2 = new_car_assignments[city1], new_car_assignments[city2]
            new_car_assignments[city1], new_car_assignments[city2] = car2, car1

    return new_route, new_car_assignments

This code calculates the total distance for a given route, car assignments, and cars.

The function `calculate_total_distance_geral` takes three parameters: `route`, `car_assignments`, and `cars`. 

The `route` parameter represents the sequence of locations to be visited. It is a list of integers.

The `car_assignments` parameter represents the assignment of cars to each location in the route. It is a list of integers.

The `cars` parameter represents the cars available for the route. It is a list of car objects.

The function initializes the `total_distance` variable to 0.

The `route` list is modified by adding a 0 at the beginning and end. This is done to represent the starting and ending location of the route.

The `current_car` variable is initialized with the car assignment for the first location in the route.

The `current_location` variable is initialized with 0, representing the starting location.

The function then iterates over each location in the route using a for loop. The loop variable `next_location` represents the next location to be visited.

Inside the loop, there is an if-else statement that checks if the current car assignment is different from the car assignment for the next location.

If the car assignments are different, the function adds the distance between the current location and the next location, as well as the return rate between the current location and the next location, to the `total_distance` variable.

If the car assignments are the same, the function only adds the distance between the current location and the next location to the `total_distance` variable.

After calculating the distance for the current location, the `current_location` variable is updated to the next location.

Finally, the function returns the `total_distance` variable, which represents the total distance for the given route, car assignments, and cars.

In [5]:


def calculate_total_distance_geral(route, car_assignments, cars):
    total_distance = 0
    route = [0] + route + [0]
    current_car = car_assignments[0]
    current_location = 0 
    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance

# simulated anealing 

This code implements the Simulated Annealing algorithm. Simulated Annealing is a metaheuristic algorithm used to solve optimization problems. It is inspired by the annealing process in metallurgy, where a material is heated and then slowly cooled to reduce defects and improve its structure.

The `simulated_annealing` function takes several parameters: `num_iterations`, `initial_temperature`, `cooling_rate`, `route`, `car_assignments`, and `cars`. 

- `num_iterations` specifies the number of iterations the algorithm will run for.
- `initial_temperature` is the initial temperature of the system. It controls the probability of accepting worse solutions.
- `cooling_rate` determines how quickly the temperature decreases over time.
- `route` is a list representing the current route.
- `car_assignments` is a list representing the current assignments of cars to customers.
- `cars` is a list of available cars.

The function initializes the current cost by calling the `calculate_total_distance` function, passing in the current `route`, `car_assignments`, and `cars`. It also initializes the `best_route`, `best_car_assignments`, and `best_cost` variables with the current values.

The algorithm then enters a loop that runs for `num_iterations` iterations. In each iteration, it calculates the current temperature based on the initial temperature and the cooling rate. It then generates a random neighbor solution by calling the `generate_random_neighbor` function, passing in the current `route` and `car_assignments`. The `generate_random_neighbor` function returns a new route and car assignments.

The algorithm calculates the cost of the neighbor solution by calling the `calculate_total_distance` function with the neighbor route, neighbor car assignments, and cars. It then checks if the neighbor solution should be accepted as the new current solution. This is determined by the `accept_solution` function, which compares the current cost and neighbor cost with the temperature. If the neighbor solution is accepted, the current route, car assignments, and cost are updated.

The algorithm also checks if the current cost is better than the best cost found so far. If it is, the best route, car assignments, and cost are updated.

After the loop finishes, the function returns the best route, car assignments, and cost.

Overall, the code implements the Simulated Annealing algorithm to find the best route and car assignments for a given problem. It iteratively explores neighbor solutions and accepts worse solutions with a certain probability based on the temperature. The algorithm gradually decreases the temperature over time, allowing it to escape local optima and potentially find better solutions.

O(n)		
The function calculate_total_distance_geral has a single for loop that iterates through the 'route' list. The number of iterations in the loop is directly proportional to the size of the 'route' list, which is represented by 'n'. Therefore, the time complexity of the code is O(n).

O(n^2)
The function simulated_annealing contains a loop that iterates 'num_iterations' times, which is proportional to the number of cities. Inside the loop, there is a call to the 'generate_random_neighbor' function, which has a time complexity of O(n). Additionally, there is a call to the 'calculate_total_distance_geral' function, which also has a time complexity of O(n). Therefore, the overall time complexity of the code snippet is O(n^2), where n is the number of cities.

c

In [6]:
import random
import math



# Função de aceitação com probabilidade
def accept_solution(current_cost, neighbor_cost, temperature):
    if neighbor_cost < current_cost:
        return True
    else:
        probability = math.exp(-(neighbor_cost - current_cost) / temperature)
        return random.random() < probability

# Simulated Annealing
def simulated_annealing(instance, cars, initial_temperature, cooling_rate):
    num_cars = instance.cars_number
    num_cities = instance.dimension
    car_assignments = [random.randint(0, num_cars - 1) for _ in range(0, num_cities)]    
    route = random.sample(range(1, num_cities), num_cities-1)
    num_iterations = num_cities * 10
    current_cost = calculate_total_distance_geral(route, car_assignments, cars)
    best_route = route
    best_car_assignments = car_assignments
    best_cost = current_cost

    for i in range(num_iterations):
        temperature = initial_temperature * math.exp(-cooling_rate * i)
        neighbor_route, neighbor_car_assignments = generate_random_neighbor(route, car_assignments)
        neighbor_cost = calculate_total_distance_geral(neighbor_route, neighbor_car_assignments, cars)
        if accept_solution(current_cost, neighbor_cost, temperature):
            route, car_assignments = neighbor_route, neighbor_car_assignments
            current_cost = neighbor_cost

        if current_cost < best_cost:
            best_route, best_car_assignments, best_cost = route, car_assignments, current_cost

    return best_route, best_car_assignments, best_cost


# Executar Simulated Annealing
best_route, best_car_assignments, best_cost = simulated_annealing(cars_instance,cars , initial_temperature= 100.0, cooling_rate= 0.02)

print("Melhor rota:", best_route)
print("Melhor alocação de carros:", best_car_assignments)
print("Melhor custo:", best_cost)


Melhor rota: [4, 1, 3, 5, 2]
Melhor alocação de carros: [0, 0, 0, 1, 1, 1]
Melhor custo: 397


# Tabu search

xThis code defines two functions for solving the Traveling Salesman Problem (TSP) with car assignment.

The objective_function function takes in a route (a list of locations), car_assignments (a list of car assignments for each location), and cars (a list of available cars). It calls the calculate_total_distance_geral function to calculate the total distance of the route, taking into account the car assignments. The objective function returns this total distance.

The get_neighbors function takes in a route and car_assignments. It initializes an empty list called neighbors. It then iterates over the locations in the route using a nested loop. In the inner loop, it swaps the positions of two cities in the route to create a new neighbor route. It appends this neighbor route, along with the original car_assignments, to the neighbors list.

Next, it iterates over the car assignments using another nested loop. In the inner loop, it swaps the car assignments of two locations in the car_assignments list to create a new neighbor car assignment. It appends the original route and this neighbor car assignment to the neighbors list.

Finally, the function returns the neighbors list, which contains all the possible neighbor solutions to the given route and car_assignments.

In [7]:
import random


# Define the objective function for TSP with car assignment
def objective_function(route, car_assignments, cars):
    return calculate_total_distance_geral(route, car_assignments, cars)

# Define the neighborhood function for TSP with car assignment
def get_neighbors(route, car_assignments):
    neighbors = []
    num_locations = len(route)

    # Swap two cities
    for i in range(num_locations):
        for j in range(i + 1, num_locations):
            neighbor_route = route[:]
            neighbor_route[i], neighbor_route[j] = neighbor_route[j], neighbor_route[i]
            neighbors.append((neighbor_route, car_assignments))

    # Swap two car assignments
    for i in range(num_locations):
        for j in range(i + 1, num_locations):
            neighbor_car_assignments = car_assignments[:]
            neighbor_car_assignments[i], neighbor_car_assignments[j] = neighbor_car_assignments[j], neighbor_car_assignments[i]
            neighbors.append((route, neighbor_car_assignments))

    return neighbors



This code implements the Tabu Search algorithm to solve a car assignment problem. The goal is to assign a set of cars to a set of cities in a way that minimizes the total distance traveled.

The algorithm starts by generating an initial solution randomly. The car assignments are represented by a list of integers, where each integer corresponds to the index of the car assigned to a city. The initial route is also generated randomly, representing the order in which the cities are visited.

The best route and car assignments are initially set to the initial solution, and the best distance is calculated using the objective function.

The algorithm then enters a loop that runs for a specified number of iterations. In each iteration, the algorithm generates a set of neighboring solutions by making small modifications to the current solution. These modifications can include swapping the car assignments of two cities or changing the order in which two cities are visited.

The algorithm evaluates each neighboring solution using the objective function and selects the best one as the next current solution. However, the algorithm also maintains a tabu list, which keeps track of recently visited solutions that should be avoided. If a neighboring solution is in the tabu list, it is not considered as a candidate for the next current solution.

After selecting the best neighboring solution, it is added to the tabu list. If the tabu list exceeds a specified size, the oldest solution is removed from the list.

Finally, if the distance of the current solution is better than the best distance found so far, the current solution becomes the new best solution.

The algorithm continues until a stopping condition is met, which can be either a maximum number of iterations or the absence of any improving solutions.

The algorithm returns the best route, car assignments, and distance found during the search.

Overall, the Tabu Search algorithm iteratively explores the solution space by considering neighboring solutions and avoiding previously visited solutions. This allows it to escape local optima and find better solutions.

In [8]:

# Define the Tabu Search algorithm for TSP with car assignment
def tabu_search(cars_instance,cars, num_iterations, tabu_list_size):
    num_cities = cars_instance.dimension
    num_cars = cars_instance.cars_number
    initial_car_assignments = [random.randint(0, num_cars - 1) for _ in range(0, num_cities)]    
    initial_route = random.sample(range(1, num_cities), num_cities-1)
    best_route = initial_route
    best_car_assignments = initial_car_assignments
    best_distance = objective_function(initial_route, initial_car_assignments, cars)

    current_route = initial_route
    current_car_assignments = initial_car_assignments
    tabu_list = []

    for _ in range(num_iterations):
        neighbors = get_neighbors(current_route, current_car_assignments)
        best_neighbor = None
        best_neighbor_distance = float('inf')

        for neighbor_route, neighbor_car_assignments in neighbors:
            if (neighbor_route, neighbor_car_assignments) not in tabu_list:
                neighbor_distance = objective_function(neighbor_route, neighbor_car_assignments, cars)
                if neighbor_distance < best_neighbor_distance:
                    best_neighbor = (neighbor_route, neighbor_car_assignments)
                    best_neighbor_distance = neighbor_distance

        if best_neighbor is None:
            # No non-tabu neighbors found, terminate the search
            break

        current_route, current_car_assignments = best_neighbor
        tabu_list.append(best_neighbor)
        if len(tabu_list) > tabu_list_size:
            # Remove the oldest entry from the tabu list if it exceeds the size
            tabu_list.pop(0)

        if objective_function(current_route, current_car_assignments, cars) < best_distance:
            # Update the best solution if the current neighbor is better
            best_route, best_car_assignments = current_route, current_car_assignments
            best_distance = objective_function(current_route, current_car_assignments, cars)

    return best_route, best_car_assignments, best_distance

num_iterations = 200
tabu_list_size = 10

best_route, best_car_assignments, best_distance = tabu_search(cars_instance,cars,  num_iterations, tabu_list_size)
print("Best route:", best_route)
print("Best car assignments:", best_car_assignments)
print("Best distance:", best_distance)


Best route: [5, 3, 2, 1, 4]
Best car assignments: [0, 0, 0, 1, 1, 0]
Best distance: 270


## Algoritmo genetico

The `generate_random_individual` function generates a random individual for the population. It takes two parameters: `num_cities` and `num_cars`. It starts by initializing the individual with a single element, 0. Then, it appends random integers between 0 and `num_cars - 1` to the individual for the remaining `num_cities - 1` elements. Finally, it returns the individual as a list of two lists: the first list represents the order of cities to visit (excluding the starting and ending city), and the second list represents the assignment of cars to each city.

The `initialize_population` function generates a population of random individuals. It takes three parameters: `pop_size`, `num_cities`, and `num_cars`. It uses a list comprehension to create a list of `pop_size` random individuals by calling the `generate_random_individual` function. Finally, it returns the population as a list of individuals.

The `evaluate_population` function evaluates the fitness of each individual in the population. It takes two parameters: `population` and `cars`. It initializes an empty list called `fitness_scores`. Then, it iterates over each individual in the population and calculates the total distance of the route by calling the `calculate_total_distance` function. The fitness score is calculated as the inverse of the total distance. The fitness score is then appended to the `fitness_scores` list. Finally, it returns the `fitness_scores` list.

The `select_parents` function selects two parents from the population based on their fitness scores. It takes two parameters: `population` and `fitness_scores`. It calculates the total fitness by summing up all the fitness scores. Then, it calculates the probabilities of selecting each individual as a parent by dividing each fitness score by the total fitness. The `random.choices` function is used to select two parents from the population based on the calculated probabilities. Finally, it returns the selected parents as a list.

The `crossover_route` function performs crossover between two routes. It takes three parameters: `list1`, `list2`, and `crossover_point`. It splits `list1` and `list2` at the `crossover_point` and creates two new lists: `left_part1` and `left_part2`. Then, it creates `result1` by concatenating `left_part1` with the elements from `list2` that are not already in `left_part1`. Similarly, it creates `result2` by concatenating `left_part2` with the elements from `list1` that are not already in `left_part2`. Finally, it returns `result1` and `result2`.

The `crossover` function performs crossover between two parents. It takes two parameters: `parent1` and `parent2`. It generates a random crossover point between 1 and the length of `parent1` - 1. Then, it calls the `crossover_route` function to perform crossover on the first list of each parent. The resulting lists are stored in `result1` and `result2`. The second list of each parent is also crossed over at the same crossover point. Finally, it returns two children as a list of two individuals.

The `mutate` function performs mutation on an individual. It takes three parameters: `individual`, `mutation_rate`, and `num_cars`. It iterates over the second list of the individual (representing the assignment of cars to cities) starting from index 1. For each element, it checks if a random number between 0 and 1 is less than the mutation rate. If it is, it assigns a random integer between 0 and `num_cars - 1` to the element. Finally, it returns the mutated individual.

Overall, these functions are used in a genetic algorithm to generate an initial population, evaluate the fitness of each individual, select parents for reproduction, perform crossover and mutation, and create a new population for the next generation.

In [9]:
import random

def calculate_total_distance(route, car_assignments, cars):
    total_distance = 0

    current_car = car_assignments[0]
    current_location = 0 
    
    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]

        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]

        current_location = next_location

    return total_distance
def generate_random_individual(num_cities, num_cars):
    individual0 = [0] 
    for _ in range(0, num_cities - 1):
        individual0.append(random.randint(0, num_cars - 1))
    return [random.sample(range(1, num_cities), num_cities-1), individual0]

def initialize_population(pop_size, num_cities, num_cars):
    population = [generate_random_individual(num_cities, num_cars) for _ in range(pop_size)]
    return population

def evaluate_population(population, cars):
    fitness_scores = []
    for individual in population:
        total_distance = calculate_total_distance([0] + individual[0] + [0], individual[1], cars)
        fitness_scores.append(1 / total_distance)  
    return fitness_scores
def select_parents(population, fitness_scores):
    total_fitness = sum(fitness_scores)
    probabilities = [score / total_fitness for score in fitness_scores]
    parents = random.choices(population, probabilities, k=2)
    return parents
def crossover_route(list1, list2, crossover_point):
    left_part1 = list1[:crossover_point]
    left_part2 = list2[:crossover_point]
    result1 = left_part1 + [x for x in list2 if x not in left_part1]
    result2 = left_part2 + [x for x in list1 if x not in left_part2]
    return result1, result2
def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    result1, result2 = crossover_route(parent1[0], parent2[0], crossover_point)
    child1 = [result1, parent1[1][:crossover_point] + parent2[1][crossover_point:]]
    child2 = [result2, parent2[1][:crossover_point] + parent1[1][crossover_point:]]
    return child1, child2
def mutate(individual, mutation_rate, num_cars):
    for i in range(1, len(individual)):
        if random.random() < mutation_rate:
            individual[1][i] = random.randint(0, num_cars - 1)
    return individual


This code implements a genetic algorithm to solve a problem related to cars and cities. 

1. The function `genetic_algorithm` takes several parameters: `num_generations` (the number of generations to run the algorithm), `pop_size` (the size of the population), `mutation_rate` (the probability of mutation), `num_cities` (the number of cities in the problem), `num_cars` (the number of cars available), and `cars` (a list of cars with their respective capacities).

2. The function starts by initializing the population using the `initialize_population` function. This function creates a random population of individuals, where each individual represents a possible solution to the problem. Each individual is a list of cities, and each city is represented by an index.

3. The `best_individual_list` is initialized as an empty list. This list will store the best individual found in each generation.

4. The algorithm then enters a loop that iterates over the specified number of generations. In each generation, the following steps are performed:

   a. The fitness scores of the population are evaluated using the `evaluate_population` function. This function calculates the fitness score for each individual in the population based on their total distance traveled.

   b. A new population is created by selecting parents from the current population, performing crossover and mutation operations, and adding the resulting children to the new population. The `select_parents` function is used to select two parents based on their fitness scores. The `crossover` function is used to create two children by combining the genetic material of the parents. The `mutate` function is used to introduce random changes (mutations) in the children's genetic material.

   c. The new population is added to the `new_population` list.

   d. The best individual in the current generation is determined by finding the individual with the highest fitness score. This is done using the `max` function with a lambda function as the key. The lambda function calculates the fitness score for each individual by calling the `calculate_total_distance` function.

   e. The best individual in the current generation is added to the `best_individual_list`.

5. After all the generations have been processed, the algorithm finds the best solution in the `best_individual_list`. This is done by finding the individual with the highest fitness score, again using the `max` function with a lambda function as the key.

6. The total distance of the best solution is calculated by calling the `calculate_total_distance` function with the best individual's cities and cars.

7. Finally, the best individual and its distance are returned as the result of the `genetic_algorithm` function.


In [10]:

def genetic_algorithm(num_generations, pop_size, mutation_rate, num_cities, num_cars, cars):
    population = initialize_population(pop_size, num_cities, num_cars)
    best_individual_list = []

    for generation in range(num_generations):
        fitness_scores = evaluate_population(population, cars)
        new_population = []
        
        for _ in range(pop_size // 2):
            parent1, parent2 = select_parents(population, fitness_scores)
            child1, child2 = crossover(parent1, parent2)
            child1 = mutate(child1, mutation_rate, num_cars)
            child2 = mutate(child2, mutation_rate, num_cars)
            new_population.extend([child1, child2])
        
        best_individual_generation = max(population, key=lambda ind: 1 / calculate_total_distance([0] + ind[0] + [0],ind[1], cars))
        best_individual_list.append(best_individual_generation)
    
    # Após as iterações, encontre a melhor solução na população
    best_individual = max(best_individual_list, key=lambda ind: 1 / calculate_total_distance([0] + ind[0] + [0],ind[1], cars))
    best_distance = calculate_total_distance([0] + best_individual[0] + [0],best_individual[1],  cars)
    
    return best_individual, best_distance

# Executando o algoritmo genético
best_solution, best_distance = genetic_algorithm(num_generations=50, pop_size=500, mutation_rate=0.2, num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)

print("Melhor rota:", best_solution)
print("Melhor distância:", best_distance)


Melhor rota: [[4, 1, 2, 5, 3], [0, 0, 0, 0, 0, 0]]
Melhor distância: 234


In [11]:
# import os

# arr = sorted(os.listdir('../PCA-main 2/CaRS_NaoEuclidianas'))

# # Initialize variables to store the best results
# best_results = {
#     'file': None,
#     'mutation_rate': None,
#     'num_generations': None,
#     'pop_size': None,
#     'best_solution': None,
#     'best_distance': float('inf')  # Initialize with a large value
# }

# for file in arr[:3]:
#     file_path = '../PCA-main 2/CaRS_NaoEuclidianas/' + file
#     cars_instance, cars = read_cars_instance(file_path)
#     for mutation_rate in [0.1, 0.2, 0.5, 0.8]:
#         for num_generations in [50, 100, 150]:
#             for pop_size in [200, 500, 800]:
#                 # Experiment with different mutation rates, num_generations, and pop_size
#                 best_solution, best_distance = genetic_algorithm(num_generations=num_generations, pop_size=pop_size, mutation_rate=mutation_rate,
#                                                                   num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)

#                 # Check if the current solution is better than the stored best solution
#                 if best_distance < best_results['best_distance']:
#                     best_results['file'] = file
#                     best_results['mutation_rate'] = mutation_rate
#                     best_results['num_generations'] = num_generations
#                     best_results['pop_size'] = pop_size
#                     best_results['best_solution'] = best_solution
#                     best_results['best_distance'] = best_distance

# # Print the best results
# print("Best Results:")
# print(f"File: {best_results['file']}")
# print(f"Mutation Rate: {best_results['mutation_rate']}")
# print(f"Num Generations: {best_results['num_generations']}")
# print(f"Pop Size: {best_results['pop_size']}")
# print("Best Solution:", best_results['best_solution'])
# print("Best Distance:", best_results['best_distance'])

# algoritmo evolutivo 

In [12]:

# Executando o algoritmo evolutivo
best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.9, num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)

print("Melhor rota:", best_solution)
print("Melhor distância:", best_distance)


Melhor rota: [[4, 1, 2, 5, 3], [0, 0, 0, 0, 0, 0]]
Melhor distância: 234


# Brute force 

This code is an algorithm to solve the Traveling Salesman Problem (TSP) with the addition of cars. The goal is to find the shortest route that visits all locations and returns to the starting location, while also assigning each location to a car.

The code begins by importing the necessary modules: `itertools` for generating permutations and combinations, and `time` for measuring the execution time.

Next, there is a function called `calculate_total_distance` that takes in the current route, car assignments, cars, and the best distance found so far. This function calculates the total distance traveled for a given route and car assignments. It iterates over each location in the route and checks if the current car assignment is different from the next location's car assignment. If they are different, it adds the distance from the current location to the next location and the return rate from the next location to the current location. If they are the same, it only adds the distance between the current and next locations. After each iteration, it checks if the current total distance is greater than the best distance found so far. If it is, it immediately returns the current total distance. Finally, it updates the current location to be the next location.

The main function is called `branch_and_bound_tsp_with_cars` and takes in an instance of the problem and a list of cars. It initializes some variables: `num_cars` to store the number of cars, `num_locations` to store the number of locations, `best_route` to store the best route found so far, and `best_distance` to store the best distance found so far (initialized as infinity).

The function then enters a nested loop using `itertools.permutations` and `itertools.product` to generate all possible routes and car assignments. It ensures that the route starts and ends at location 0 by adding it to the beginning and end of the route. For each combination of route and car assignments, it calls the `calculate_total_distance` function to calculate the total distance traveled. If the calculated distance is less than the best distance found so far, it updates the best distance, best route, and best car assignments.

After the nested loop, the function returns the best route, best distance, and best car assignments.

The code then measures the execution time by recording the start time using `time.time()`. It calls the `branch_and_bound_tsp_with_cars` function with the given instance and cars, and stores the returned values in `best_route`, `best_distance`, and `best_car_assignments`. It records the end time and calculates the execution time by subtracting the start time from the end time.

Finally, it prints the execution time, best route, best distance, and best car assignments.

In [13]:
import itertools
import time

def calculate_total_distance_brute_force(route, car_assignments, cars, best_distance):
    total_distance = 0
    current_car = car_assignments[0]
    current_location = 0  # Comece em qualquer cidade (0 é geralmente a primeira cidade)

    for next_location in route:
        if current_car != car_assignments[next_location]:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
            total_distance += cars[current_car].return_rate_matrix[current_location][next_location]
        else:
            total_distance += cars[current_car].distance_matrix[current_location][next_location]
        if best_distance < total_distance:
            return total_distance

        current_location = next_location

    return total_distance

def brute_force_tsp_with_cars(instance, cars):
    num_cars = instance.cars_number
    num_locations = instance.dimension

    best_route = None
    best_distance = float('inf')

    for route in itertools.permutations(range(1, num_locations), num_locations - 1):
        for car_assignments in itertools.product(range(num_cars), repeat=num_locations):
            # Certifique-se de que a rota começa e termina na cidade 0
            full_route = (0,) + route + (0,)

            distance = calculate_total_distance_brute_force(full_route, car_assignments, cars, best_distance)

            if distance < best_distance:
                best_distance = distance
                best_route = full_route
                best_car_assignments = car_assignments

    return best_route, best_distance, best_car_assignments

start_time = time.time()

best_route, best_distance, best_car_assignments = brute_force_tsp_with_cars(cars_instance, cars)

end_time = time.time()
print("Tempo de execução:", end_time - start_time, "segundos")

print("Melhor rota:", best_route)
print("Melhor distância:", best_distance)
print("best_car_assignments:", best_car_assignments)



Tempo de execução: 0.008733034133911133 segundos
Melhor rota: (0, 4, 1, 2, 3, 5, 0)
Melhor distância: 213
best_car_assignments: (0, 0, 0, 0, 0, 0)


In [14]:
import os
import time
import numpy as np
from tabulate import tabulate

arr = sorted(os.listdir('../PCA-main 2/CaRS_NaoEuclidianas'))[15:]

# Define headers for the table
headers = ["File", "Alg", "City", "Car", "#Best", "T",  "std_dev" , "Worse", "Avg", "Best", "Freq"]
print(headers)

        


# Initialize a list to store the data for each row
with open("result", 'w') as arq:
    table_data = []
    
    for file in arr:
        file_path = '../PCA-main 2/CaRS_NaoEuclidianas/' + file
        cars_instance, cars = read_cars_instance(file_path)
        best_distance_all = float('inf')
        for algorithm in ["genetic_algorithm", "evolutionary_algorithm", "tabu_search", "simulated_annealing"]:
            best_distances = []
            best_costs = []
            execution_times = []

            for i in range(1, 51):  # 30 independent executions
                # Perform the selected algorithm
                start_time = time.time()
                if algorithm == "genetic_algorithm":
                    best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.9,
                                                                    num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)
                elif algorithm == "evolutionary_algorithm":
                    best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.1,
                                                                    num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)
                elif algorithm == "tabu_search":
                    best_route, best_car_assignments, best_cost = tabu_search(cars_instance, cars, num_iterations=100, tabu_list_size=10)
                    best_distance = best_cost  
                elif algorithm == "simulated_annealing":
                    best_route, best_car_assignments, best_cost = simulated_annealing(cars_instance, cars, initial_temperature=150.0, cooling_rate=0.04)
                    best_distance = best_cost  
                end_time = time.time()

                # Calculate metrics
                best_distances.append(best_distance)
                execution_times.append(end_time - start_time)
                
                arq.write(str([file, algorithm, cars_instance.dimension, cars_instance.cars_number, i , best_distance, end_time - start_time]) + "\n")

                # Append data for the current execution to the table_data list
            table_data.append([file, algorithm, cars_instance.dimension, cars_instance.cars_number, 0, 0, 0, 0, 0, 0, 0])

            # Calculate additional metrics for the current algorithm and file
            avg_time = np.mean(execution_times)
            avg_distance = np.mean(best_distances)
            std_dev_distance = np.std(best_distances)
            worst_distance = np.max(best_distances)
            best_distance = np.min(best_distances)
            frequency_distance = best_distances.count(best_distance)
    
            best_distance_all = best_distance if best_distance < best_distance_all else best_distance_all

            # Update table_data with the calculated metrics
            for row in table_data:
                if row[0] == file and row[1] == algorithm and row[2] == cars_instance.dimension and row[3] == cars_instance.cars_number:
                    row[4] = best_distance_all
                    row[5] = round(avg_time, 3)
                    row[6] = round(std_dev_distance, 3)
                    row[7] = worst_distance
                    row[8] = round(avg_distance , 3)
                    row[9] = best_distance
                    row[10] = frequency_distance
                if row[0] == file and row[2] == cars_instance.dimension and row[3] == cars_instance.cars_number:
                    if best_distance_all < row[4]:
                        row[4] = best_distance_all
            arq.write(str(row) +  "\n")
        for row in table_data:
            print(row)
    arq.write("EOF\n")

# Print the table using the tabulate function
print(tabulate(table_data, headers=headers))


['File', 'Alg', 'City', 'Car', '#Best', 'T', 'std_dev', 'Worse', 'Avg', 'Best', 'Freq']
['BrasilPR25n.car', 'genetic_algorithm', 25, 3, 613, 1.511, 19.182, 1017, 978.84, 931, 1]
['BrasilPR25n.car', 'evolutionary_algorithm', 25, 3, 613, 1.365, 23.069, 1013, 970.34, 921, 1]
['BrasilPR25n.car', 'tabu_search', 25, 3, 613, 0.162, 33.518, 757, 679.92, 613, 1]
['BrasilPR25n.car', 'simulated_annealing', 25, 3, 613, 0.004, 107.275, 1043, 838.74, 684, 1]
['BrasilPR25n.car', 'genetic_algorithm', 25, 3, 613, 1.511, 19.182, 1017, 978.84, 931, 1]
['BrasilPR25n.car', 'evolutionary_algorithm', 25, 3, 613, 1.365, 23.069, 1013, 970.34, 921, 1]
['BrasilPR25n.car', 'tabu_search', 25, 3, 613, 0.162, 33.518, 757, 679.92, 613, 1]
['BrasilPR25n.car', 'simulated_annealing', 25, 3, 613, 0.004, 107.275, 1043, 838.74, 684, 1]
['BrasilRJ14n.car', 'genetic_algorithm', 14, 2, 230, 1.227, 8.528, 291, 277.0, 254, 1]
['BrasilRJ14n.car', 'evolutionary_algorithm', 14, 2, 230, 1.207, 9.82, 296, 276.86, 256, 1]
['BrasilRJ1

In [ ]:
for citys in range(3,15):
    for cars_number in range(2,5):
        for i in range(0 ,3):
            generate_cars_instance_file(citys,cars_number, "teste.car")
            print("\hline")

            file_path = "teste.car" 
            cars_instance, cars = read_cars_instance(file_path)

            start_time1 = time.time()
            best_route, best_distance, best_car_assignments = brute_force_tsp_with_cars(cars_instance, cars)
            end_time1 = time.time()
            print( "brute\_force" ," & ",citys, " - ",cars_number ," & ", i, " & ", end_time1 - start_time1 , " & ", best_distance, "  \\\\")

  
            start_time2 = time.time()
            best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.9, num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)
            end_time2 = time.time()

            print( "genetic\_algorithm" ," & ",citys, " - ",cars_number ," & ", i, " & ", end_time2 - start_time2 , " & ", best_distance, "  \\\\")
        


            start_time3 = time.time()
            best_solution, best_distance = genetic_algorithm(num_generations=100, pop_size=500, mutation_rate=0.1, num_cities=cars_instance.dimension, num_cars=cars_instance.cars_number, cars=cars)
            end_time3 = time.time()

            print( "evolucionary\_algorithm" ," & ",citys, " - ",cars_number ," & ", i, " & ", end_time3 - start_time3 , " & ", best_distance, "  \\\\")
        

            start_time4 = time.time()
            best_route, best_car_assignments, best_cost = tabu_search(cars_instance, cars, num_iterations= 100, tabu_list_size= 10)
            end_time4 = time.time()

            print( "tabu\_search" ," & ",citys, " - ",cars_number ," & ", i, " & ", end_time4 - start_time4 , " & ", best_cost, "  \\\\")
        

            start_time5 = time.time()
            best_route, best_car_assignments, best_cost = simulated_annealing(cars_instance, cars, initial_temperature= 150.0, cooling_rate= 0.04)
            end_time5 = time.time()

            print( "simulated\_annealing" ," & ",citys, " - ",cars_number ," & ", i, " & ", end_time5 - start_time5 , " & ", best_cost, "  \\\\")
        
        print("\hline")

\hline
brute\_force  &  3  -  2  &  0  &  4.291534423828125e-05  &  122   \\
genetic\_algorithm  &  3  -  2  &  0  &  1.5162649154663086  &  122   \\
evolucionary\_algorithm  &  3  -  2  &  0  &  1.212285041809082  &  122   \\
tabu\_search  &  3  -  2  &  0  &  3.719329833984375e-05  &  134   \\
simulated\_annealing  &  3  -  2  &  0  &  9.298324584960938e-05  &  122   \\
\hline
brute\_force  &  3  -  2  &  1  &  2.002716064453125e-05  &  85   \\
genetic\_algorithm  &  3  -  2  &  1  &  1.2222719192504883  &  85   \\
evolucionary\_algorithm  &  3  -  2  &  1  &  1.1913321018218994  &  85   \\
tabu\_search  &  3  -  2  &  1  &  3.790855407714844e-05  &  140   \\
simulated\_annealing  &  3  -  2  &  1  &  8.511543273925781e-05  &  85   \\
\hline
brute\_force  &  3  -  2  &  2  &  1.8835067749023438e-05  &  57   \\
genetic\_algorithm  &  3  -  2  &  2  &  1.6320371627807617  &  57   \\
evolucionary\_algorithm  &  3  -  2  &  2  &  2.1010470390319824  &  57   \\
tabu\_search  &  3  -  2  &

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/_2/44xw3xz51713xrskxxplzf_00000gn/T/ipykernel_4829/3297334335.py", line 11, in <module>
    best_route, best_distance, best_car_assignments = brute_force_tsp_with_cars(cars_instance, cars)
                                                      ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/var/folders/_2/44xw3xz51713xrskxxplzf_00000gn/T/ipykernel_4829/1121483845.py", line -1, in brute_force_tsp_with_cars
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/site-packages/IPython/core/interactiveshell.py", line 2120, in showtraceback
    stb = self.InteractiveTB.structured_traceback(
          ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.1